In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import pydicom
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
import tensorflow as tf
import random
import sklearn
import scipy

In [ ]:
# Load libraries
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
# Algorithms
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB

Read Data

In [ ]:
train_df = pd.read_csv("../input/osic-pulmonary-fibrosis-progression/train.csv")
train_df.drop_duplicates(keep=False, inplace=True, subset=['Patient','Weeks'])

test_df = pd.read_csv("../input/osic-pulmonary-fibrosis-progression/test.csv")

sub_df = pd.read_csv("../input/osic-pulmonary-fibrosis-progression/sample_submission.csv")
sub_df['Patient'] = sub_df['Patient_Week'].apply(lambda x:x.split('_')[0])
sub_df['Weeks'] = sub_df['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
#sub_df =  sub_df[['Patient','Weeks','Confidence','Patient_Week']]
sub_test_df = sub_df.merge(test_df.drop('Weeks', axis=1), on="Patient")

#print(train_df.head())
print('Train shape: ', train_df.shape)
print('Number of unique patients : {}'.format(train_df['Patient'].nunique()))

print(test_df.head())
print('Test shape:', test_df.shape)
#print('Number of patiens in test: {}'.format(test_df['Patient'].nunique()))

print('Submission shape: ',sub_df.shape)
print('Number of patients in submission : {}'.format(sub_df['Patient'].nunique()))
#print(sub_test_df.head())

In [ ]:
train_df['FVC'].hist()
#this is a skewed histograph, let's see if each patient's change in FVC is also a skewd distribution

In [ ]:
#train_df['ChangeFVC'].hist()

# Analysis temporal trends of FVC development

In [ ]:
# Plot time v.s. FVC trend for each patient

train_df["Patient"].nunique()
Plist = list(train_df.Patient.unique())
PID='ID00007637202177411956430'
#train_df.loc[train_df.Patient == PID]

pat=train_df.groupby(['Patient'])['FVC'].count().to_frame().reset_index()
print(pat.head())

Week = np.arange(-12, 134)
Plist = list(train_df.Patient.unique())
train_df2 = pd.DataFrame(Week, columns = ["Weeks"])
train_df2.insert(1, 'FVC', np.nan)
train_df2.insert(2, 'Percent', np.nan)
train_df2.insert(3, 'Age', np.nan)
train_df2.insert(4, 'Sex', np.nan)
train_df2.insert(5, 'SmokingStatus', np.nan)


# highlight a specific patient
PID = train_df.loc[train_df.Patient == Plist[3]]
PID = PID.reset_index()
print(train_df2.head())

for i, D in enumerate(PID.Weeks):
    D = D + 12
    train_df2.at[D, "FVC"] = PID.FVC[i]
    train_df2.at[D, "Percent"] = PID.Percent[i]

train_df2.loc[:, "Age"] = PID.Age[0]
train_df2.loc[:, "Sex"] = PID.Sex[0]
train_df2.loc[:, "SmokingStatus"] = PID.SmokingStatus[0]
    
train_df2 = train_df2.interpolate('linear', order=2, limit_direction='both')
train_df2
#view the FVC v.s. Weeks for one specific patient
plt.figure(figsize=(18,6))
grp = train_df

plt.xlabel("Weeks")
plt.ylabel("FVC")
plt.plot(grp.Weeks, grp.FVC, marker="x")
plt.plot(PID.Weeks, PID.FVC, marker="o", markersize=8)

Plist=pd.DataFrame(Plist)
print(Plist)


Time progression
Weeks v.s. delta FVC%

Train dataset

In [ ]:
# set a base level (1'st week) for all data
train_base = train_df.drop_duplicates(subset='Patient', keep='first')

train_base = train_base[['Patient', 'Weeks', 'FVC', 
                   'Percent', 'Age']].rename(columns={'Weeks': 'BaseWeek',
                                                      'Percent': 'BasePercent',
                                                      'Age': 'BaseAge',
                                                      'FVC': 'BaseFVC'})

train_df = pd.merge(train_df,train_base,on='Patient',how='left')
print(train_df.head())



In [ ]:
#set a last data (the train_last is not used in fitting, this is just to view the change of FVC in the final week)
#view a histgram of percentage change
train_last = train_df.drop_duplicates(subset='Patient', keep='last')
train_last = train_last[['Patient', 'Weeks', 'FVC', 
                   'Percent', 'Age']].rename(columns={'Weeks': 'LastWeek',
                                                      'Percent': 'LastPercent',
                                                      'Age': 'LastAge',
                                                      'FVC': 'LastFVC'})
train_last = pd.merge(train_df,train_last,on='Patient',how='left')


train_last['lastChangeFVC'] = (train_last['LastFVC']-train_last['BaseFVC'])/train_last['BaseFVC']
train_last['TimePassed'] = (train_last['LastWeek']-train_last['Weeks'])
train_last = train_last.loc[train_last['TimePassed']==0,:]
print(train_last.head())
train_last["lastChangeFVC"].hist()

In [ ]:
from pylab import *
from scipy.optimize import curve_fit

data=train_last["lastChangeFVC"]
y,x,_=hist(data,label='data')

x=(x[1:]+x[:-1])/2 # for len(x)==len(y)

def gauss(x,mu,sigma,A):
    return A*exp(-(x-mu)**2/2/sigma**2)

def bimodal(x,mu1,sigma1,A1,mu2,sigma2,A2):
    return gauss(x,mu1,sigma1,A1)+gauss(x,mu2,sigma2,A2)

expected=(0,0.1,4,-0.1,0.1,4)
params,cov=curve_fit(bimodal,x,y,expected)
sigma=sqrt(diag(cov))
plot(x,bimodal(x,*params),color='red',lw=3,label='model')
legend()
print(params,'\n',sigma)  
print(params)  
# via pandas :
# pd.DataFrame(data={'params':params,'sigma':sigma},index=bimodal.__code__.co_varnames[1:])



In [ ]:
#later split the dataset into two classes, set the threshold at 0.03

train_last["class"]=0

##class1 <= -0.03
#id1=train_last.index[train_last["lastChangeFVC"]<=-0.03]
#id2=train_last.index[train_last["lastChangeFVC"]>=-0.03]

#class1 <= 0
id1=train_last.index[train_last["lastChangeFVC"]<=0]
id2=train_last.index[train_last["lastChangeFVC"]>=0]
train_last.at[id2,"class"]=1



Class1ID=train_last.loc[id1,"Patient"]
Class2ID=train_last.loc[id2,"Patient"]
print(shape(Class1ID))
shape(train_last)
train_last=train_last.iloc[:,np.r_[0,15,17]]
print(train_last.head())

In [ ]:
#later split the dataset into two classes, set the threshold at 0.03

In [ ]:
# calculate change for FVC and time

train_df = train_df.merge(train_last, on="Patient")
print(train_df.head())
train_df['Visit'] = 1
train_df['Visit'] = train_df[['Patient', 'Visit']].groupby('Patient').cumsum()
train_df['ChangeFVC'] = (train_df['FVC']-train_df['BaseFVC'])/train_df['BaseFVC']
train_df['TimePassed'] = (train_df['Weeks']-train_df['BaseWeek'])
print(train_df.head(150))

In [ ]:
#remove base line data

train_df = train_df.loc[train_df['TimePassed']>0,:]

train_df = pd.get_dummies(train_df, columns=['Sex'])
train_df = pd.get_dummies(train_df, columns=['SmokingStatus'])
train_df.head()

In [ ]:
#later split the dataset into two classes, set the threshold at 0.03

train_df.head()

In [ ]:
#Get rid of redundant data
train_input=train_df.iloc[:,np.r_[0,1,2:4,6,7,10,12:19]]
train_input.head()

In [ ]:
train_input.describe()

In [ ]:
# Generate a mask for the upper triangle (taken from seaborn example gallery)
# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style
corr=train_input.corr()

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
plt.subplots(figsize = (12,8))
sns.heatmap(corr, 
            annot=True,
            mask = mask,
            cmap = 'RdBu',
            linewidths=.9, 
            linecolor='white',
            vmax = 0.3,
            fmt='.2f',
            center = 0,
            square=True)
plt.title("Correlations Matrix", y = 1,fontsize = 20, pad = 20);

In [ ]:
#get a correlation matrix with Change in FVC (%)
pd.DataFrame(abs(train_input.corr()['ChangeFVC']).sort_values(ascending = False))

In [ ]:
##get a correlation matrix with FVC
pd.DataFrame(abs(train_input.corr()['FVC']).sort_values(ascending = False))

In [ ]:
train_input.to_csv("class.csv")
Class1ID.to_csv('PatientID.csv')

# Predictive Modeling with sklearn Models (LR, LDA, KNN, CART, NB, SVM)


In [ ]:
#FVC	Percent	BaseFVC	BasePercent	ChangeFVC	TimePassed	Sex_Female	Sex_Male	SmokingStatus_Currently smokes	SmokingStatus_Ex-smoker	SmokingStatus_Never smoked
X = train_input.iloc[:,np.r_[1,2,3,4,6:11]]


#change float to interger as the models does not work on floats
#y = train_input.iloc[:,1]  #FVC
y = train_input.iloc[:,4]*1000 #ChangeFVC
y = train_input.iloc[:,1]*10 #Percent
y =  [round(x) for x in y]
y = pd.DataFrame(y,columns=['ChangeFVC'])
#y = train_input.iloc[:,1]

X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.10, random_state=1)  #test accuracy using 10% of the sample size
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.10, random_state=0)  #test accuracy using 10% of the sample size


print(X.head())
print(y.head())
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

In [ ]:

# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))


In [ ]:
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=3, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    
    

In [ ]:
# Compare Algorithms
pyplot.boxplot(results, labels=names)
pyplot.title('Algorithm Comparison')
pyplot.show()
1
2
3
4
# Compare Algorithms
pyplot.boxplot(results, labels=names)
pyplot.title('Algorithm Comparison')
pyplot.show()

In [ ]:

# Make predictions on validation dataset
model = SVC(gamma='auto')
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

In [ ]:
# Evaluate predictions
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
#  TAT

Random Forest